# Bank Customer Segmentation

## Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [ ]:
bank_cust = pd.read_csv('/kaggle/input/bank-customer-segmentation/bank_transactions.csv')
bank_cust.head()

In [ ]:
bank_cust.info()

In [ ]:
bank_cust.describe()

## Exploratory Data Analysis

In [ ]:
bank_eda = bank_cust.copy()

### Any duplicated values?

In [ ]:
bank_eda.duplicated().sum()

### Any NULL values?

In [ ]:
null_values = bank_eda.isnull().sum()

In [ ]:
null_values.values

In [ ]:
null_values.plot(kind='bar',cmap='viridis')
plt.xlabel('Columns')
plt.ylabel('Count of Null Values')
plt.show()

### Counts of Categorical values?

In [ ]:


sns.countplot(data=bank_eda,x='CustGender')
plt.show()

print(bank_eda['CustLocation'].value_counts())
    

### Boxplot for Numerical Columns

In [ ]:
num_columns = ['CustAccountBalance','TransactionAmount (INR)']

for col in num_columns:
    sns.boxplot(data=bank_eda,y=col)
    plt.show()

### Boxplot of Account Balance and Transaction Amount by Gender

In [ ]:
for col in num_columns:
    sns.boxplot(data=bank_eda,x='CustGender',y=col)
    plt.show()

In [ ]:
(bank_eda['CustLocation'].value_counts()).sort_values(ascending=False).iloc[:10]

### Year of Transaction and Customer Age as per Transaction Year

In [ ]:
from datetime import datetime,date

bank_eda['TransactionDate'] = pd.to_datetime(bank_eda['TransactionDate'],format='mixed')
bank_eda['TransactionDate'] = bank_eda['TransactionDate'].apply(lambda x: x - pd.DateOffset(years=100) if x > datetime.now() else x)

In [ ]:
bank_eda.head()

In [ ]:
bank_eda['TransactionYear'] = bank_eda['TransactionDate'].dt.year
bank_eda.head()

In [ ]:
bank_eda['CustomerDOB'] = pd.to_datetime(bank_eda['CustomerDOB'],format='mixed')
bank_eda['CustomerDOB'] = bank_eda['CustomerDOB'].apply(lambda x: x - pd.DateOffset(years=100) if x > datetime.now() else x)
bank_eda['CustomerYear'] = bank_eda['CustomerDOB'].dt.year
bank_eda.head()

In [ ]:
bank_eda['CustomerAge'] = bank_eda['TransactionYear'] - bank_eda['CustomerYear']
bank_eda.head()

In [ ]:
bank_eda[bank_eda.CustomerAge > 200]

Outliers!?

In [ ]:
bank_eda.drop('CustomerDOB',axis=1,inplace=True)

In [ ]:
bank_eda.drop('CustomerYear',axis=1,inplace=True)

### Looks like there are Outliers

In [ ]:
sns.histplot(data=bank_eda,x='CustomerAge',bins=20)
plt.show()

In [ ]:
bank_eda[bank_eda.CustomerAge > 200]

In [ ]:
bank_clean = bank_eda[bank_eda.CustomerAge < 100]

Removed the outliers

In [ ]:
sns.histplot(data=bank_clean,x='CustomerAge',bins=20)
plt.title('Customer Age Histogram')
plt.show()

Majority customers are in the age group of 20-40

In [ ]:
bank_clean.isna().sum()

### Checking if Account Balance > Transaction Amount

In [ ]:
bank_clean[(bank_clean['CustAccountBalance'] - bank_clean['TransactionAmount (INR)'])<0]

In [ ]:
bank_clean_2 = bank_clean[(bank_clean['CustAccountBalance'] - bank_clean['TransactionAmount (INR)'])>0]

In [ ]:
bank_clean_2.isna().sum()

In [ ]:
bank_clean_2['CustGender'].fillna('M',inplace=True)
bank_clean_2['CustLocation'].fillna('MUMBAI',inplace=True)

In [ ]:
bank_clean_2.isna().sum()

### Transactions Amount and Transaction Count by Month -2016

In [ ]:
bank_clean_2['TransactionMonth'] = bank_clean_2['TransactionDate'].dt.month
bank_clean_2.head()

In [ ]:
transactions_num = bank_clean_2[['TransactionAmount (INR)','TransactionMonth']]

In [ ]:
monthly_transactions = transactions_num.groupby('TransactionMonth').agg(['median','count'])

In [ ]:
monthly_transactions

### Median Transaction Amount by Month

In [ ]:
monthly_transactions['TransactionAmount (INR)','median'].plot(kind='line')
plt.title('Median Transaction Amount by Months(1-12)')
plt.xlabel('Month')
plt.ylabel('Median Transaction Amount')
plt.show()

### Number of Transactions by Month

In [ ]:
monthly_transactions['TransactionAmount (INR)','count'].plot(kind='line')
plt.title('Number of Transactions by Months(1-12)')
plt.xlabel('Month')
plt.ylabel('Count of Transactions')
plt.show()

### Transactions by Time of Day

In [ ]:
bank_clean_2['TransactionTime'] = bank_clean_2['TransactionTime'].apply(lambda x : datetime.utcfromtimestamp(int(x)).strftime('%H:%M:%S'))

In [ ]:
bank_clean_2.head()

In [ ]:
time_split = bank_clean_2['TransactionTime'].str.split(':',expand=True)
bank_clean_2['TransactionHour'] = time_split[0].astype(int)

In [ ]:
bank_clean_2.head()

In [ ]:
transactions_num_2 = bank_clean_2[['TransactionAmount (INR)','TransactionHour']]

hourly_transactions = transactions_num_2.groupby('TransactionHour').agg(['median','count'])
hourly_transactions

### Median Transaction Amount by Time of Day

In [ ]:
hourly_transactions['TransactionAmount (INR)','median'].plot(kind='line')
plt.title('Median Transaction Amount by Time of Day')
plt.xlabel('Day Hour')
plt.ylabel('Median Transaction Amount')
plt.show()

### Number of Transactions by Time of Day

In [ ]:
hourly_transactions['TransactionAmount (INR)','count'].plot(kind='line')
plt.title('Number of Transactions by Time of Day')
plt.xlabel('Day Hour')
plt.ylabel('Count of Transactions')
plt.show()

### Transactions by Month and Time of Day

In [ ]:
transactions_3 = bank_clean_2[['TransactionAmount (INR)','TransactionMonth','TransactionHour']]

monthly_and_hourly = transactions_3.groupby(['TransactionMonth','TransactionHour']).agg(['median','count'])
monthly_and_hourly

In [ ]:
monthly_and_hourly.index

In [ ]:
months = list(range(1,13))
hours = list(range(0,24))
months

### Median Transaction Amount in Every Month and Time of Day

In [ ]:
for month in months:
    mini_month = monthly_and_hourly.loc[month]
    mini_month['TransactionAmount (INR)','median'].plot(kind='line')
    plt.title(f'Median Transaction Amount for Month {month} by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Median Transaction Amount in INR')
    plt.show()

### Number of Transactions for Every Month and Time of Day

In [ ]:
for month in months:
    mini_month = monthly_and_hourly.loc[month]
    mini_month['TransactionAmount (INR)','count'].plot(kind='line')
    plt.title(f'Number of Transactions for Month {month} by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Number of Transactions')
    plt.show()

### Correlation between Numerical Features

In [ ]:
transactions_corr = bank_clean_2[['TransactionAmount (INR)','CustAccountBalance','TransactionMonth','TransactionHour']]

corr_mat = transactions_corr.corr()

sns.heatmap(corr_mat,annot=True)
plt.show()

## Principal Component Analysis

In [ ]:
bank = bank_clean_2.copy()
bank.head()

In [ ]:
bank.drop(['TransactionID','CustomerID','CustLocation','TransactionDate','TransactionTime','TransactionYear'],axis=1,inplace=True)

In [ ]:
bank.head()

In [ ]:
bank.drop(['TransactionMonth','TransactionHour'],axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

num_pipeline = make_pipeline(StandardScaler())
cat_pipeline = make_pipeline(OneHotEncoder())
num_attribs = ['CustAccountBalance','TransactionAmount (INR)','CustomerAge']
cat_attribs = ['CustGender']
preprocessing = ColumnTransformer([
    ('num',num_pipeline,num_attribs),('cat',cat_pipeline,cat_attribs)
])

In [ ]:
bank_df = preprocessing.fit_transform(bank)

In [ ]:
print('Shape of Bank data before PCA:',bank_df.shape)

In [ ]:
from sklearn.decomposition import TruncatedSVD

pca_bank = TruncatedSVD(n_components=4)
pca_bank.fit(bank_df)

bank_transformed = pca_bank.transform(bank_df)
print('Shape of Bank data after PCA:',bank_transformed.shape)

### Plot of Principal Component-2 vs Principal Component-1

In [ ]:
plt.scatter(bank_transformed[:,0],bank_transformed[:,1])
plt.title('PCA on Bank Customer')
plt.xlabel('Principal Component-1')
plt.ylabel('Principal Component-2')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

cluster = list(range(1,21))
inertias = []
for clusters in cluster:
    kmeans_bank = KMeans(n_clusters=clusters,n_init='auto',random_state=42)
    kmeans_bank.fit(bank_transformed)
    inertias.append(kmeans_bank.inertia_)

### Knee-Elbow Plot

In [ ]:
plt.plot(cluster,inertias)
plt.xlabel('Number of Clusters')
plt.ylabel('Intertia')
plt.title('Knee-Elbow plot')
plt.grid()
plt.show()

### Principal Components Plot with Labels

In [ ]:
kmeans_chosen = KMeans(n_clusters=5,n_init='auto',random_state=42)
kmeans_chosen.fit(bank_transformed)

plt.scatter(bank_transformed[:,0],bank_transformed[:,1],c=kmeans_chosen.labels_)
plt.title('KMeans with n_clusters=5')
plt.xlabel('Principal Component-1')
plt.ylabel('Principal Component-2')
plt.show()

In [ ]:
bank['Labels'] = kmeans_chosen.labels_

In [ ]:
bank['Labels'].value_counts()

### Summary Statistics of Clusters

In [ ]:
labels = [0,1,2,3,4]
for label in labels:
    print(f'Cluster no:{label}')
    df_cluster = bank[bank.Labels == label]
    mode_gender = df_cluster['CustGender'].mode()
    mean_accountbalance = df_cluster['CustAccountBalance'].mean()
    mean_transaction_amt = df_cluster['TransactionAmount (INR)'].mean()
    mean_customer_age = df_cluster['CustomerAge'].mean()
    print(f'Most frequent gender:{mode_gender}')
    print(f'Mean Account Balance:{round(mean_accountbalance)}')
    print(f'Mean Transaction Amount (INR):{round(mean_transaction_amt)}')
    print(f'Mean Customer Age:{round(mean_customer_age)}')

1. Cluster 0 : Middle Aged Males who are probably working professionals 
2. Cluster 1: Senior Males who are well established with a high Account Balance
3. Cluster 2: Single Males who are starting out their careers with a relatively low Bank Balance and Transaction Amount
4. Cluster 3: Similar characteristics to Cluster 0
5. Cluster 4: Simialr characteristics to Cluster 0